# Data Extraction - Azure AI Document Intelligence + Azure OpenAI GPT-4o

This sample demonstrates how to extract structured data from any document using Azure AI Document Intelligence and Azure OpenAI GPT models.

![Data Extraction](../../../images/extraction-document-intelligence-openai.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes specific extraction instruction for the type of document, and the Markdown content of the document.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from a document using Azure OpenAI's GPT-4o model.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **azure-ai-documentintelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **openai** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local modules are also used:

- **modules.app_settings** to access environment variables from the `.env` file.
- **modules.comparison** to compare the output of the extraction process with expected results.
- **modules.document_intelligence_confidence** to evaluate the confidence of the extraction process based on the extracted structured output and the analysis result from Azure AI Document Intelligence.
- **modules.document_processing_result** to store the results of the extraction process as a file.
- **modules.invoice** to provide the expected structured output JSON schema for invoice documents.
- **modules.stopwatch** to measure the end-to-end execution time for the classification process.

In [19]:
import sys
sys.path.append('../../') # Import local modules

from IPython.display import display, Markdown
import os
import pandas as pd
import json
from dotenv import dotenv_values
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

from modules.app_settings import AppSettings
from modules.comparison import extraction_comparison
from modules.document_intelligence_confidence import evaluate_confidence
from modules.document_processing_result import DataExtractionResult
from modules.invoice import Invoice, InvoiceEvaluator
from modules.stopwatch import Stopwatch

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create client instances using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [20]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-08-01-preview" # Requires the latest API version for structured outputs.
)

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=settings.ai_services_endpoint,
    credential=credential
)

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on the details of an [Invoice](../../assets/invoices/invoice_1.pdf).

> **Note**: More invoice examples can be found in the [assets folder](../../assets/invoices). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

```json
{
    "fname": "<name of the invoice file>",
    "expected": {
        "customer_name": "",
        "customer_address": {
            "street": "",
            "city": "",
            "state": "",
            "postal_code": "",
            "country": ""
        },
        "customer_tax_id": "",
        "shipping_address": "",
        "purchase_order": "",
        "invoice_id": "",
        "invoice_date": "",
        "payable_by": "",
        "vendor_name": "",
        "vendor_address": "",
        "vendor_tax_id": "",
        "remittance_address": "",
        "subtotal": 0,
        "total_discount": 0,
        "total_tax": 0,
        "invoice_total": 0,
        "payment_terms": "",
        "items": [
            {
                "product_code": "",
                "description": "",
                "quantity": 0,
                "tax": 0,
                "tax_rate": "",
                "unit_price": 0,
                "total": 0,
                "reason": null
            }
        ],
        "total_item_quantity": 0,
        "items_customer_signature": {
            "signatory": "",
            "is_signed": true
        },
        "items_vendor_signature": {
            "signatory": "",
            "is_signed": true
        },
        "returns": [
            {
                "product_code": "",
                "description": "",
                "quantity": 0,
                "tax": null,
                "tax_rate": null,
                "unit_price": null,
                "total": null,
                "reason": ""
            }
        ],
        "total_return_quantity": 0,
        "returns_customer_signature": {
            "signatory": "",
            "is_signed": true
        },
        "returns_vendor_signature": {
            "signatory": "",
            "is_signed": true
        }
    },
    "max_tokens": 4096,
    "temperature": 0.1,
    "top_p": 0.1
}
```

The expected output has been defined by a human evaluating the document.

In [21]:
path = f"{working_dir}/samples/assets/invoices/"
metadata_fname = "invoice_2.json" # Change this to the file you want to evaluate
metadata_fpath = f"{path}{metadata_fname}"

with open(metadata_fpath, "r") as f:
    data = json.load(f)
    
expected = Invoice(**data['expected'])
pdf_fname = data['fname']
pdf_fpath = f"{path}{pdf_fname}"

invoice_evaluator = InvoiceEvaluator(expected)

## Extract data from the document

The following code block executes the data extraction process using Azure AI Document Intelligence and Azure OpenAI's GPT-4o model.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the Markdown.

In [22]:
with Stopwatch() as di_stopwatch:
    with open(pdf_fpath, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout",
            analyze_request=f,
            output_content_format=ContentFormat.MARKDOWN,
            content_type="application/pdf"
        )
    
    result: AnalyzeResult = poller.result()

markdown = result.content

In [23]:
with Stopwatch() as oai_stopwatch:
    completion = openai_client.beta.chat.completions.parse(
        model=settings.gpt4o_model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that extracts data from documents.",
            },
            {
                "role": "user",
                "content": f"""Extract the data from this invoice. 
                - If a value is not present, provide null.
                - Dates should be in the format YYYY-MM-DD.""",
            },
            {
                "role": "user",
                "content": markdown,
            }
        ],
        response_format=Invoice,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.1
    )

### Understanding the Structured Outputs JSON schema

Using [Pydantic's JSON schema feature](https://docs.pydantic.dev/latest/concepts/json_schema/), the [Invoice](../../modules/invoice.py) data model is automatically converted to a JSON schema when applied to the `response_format` parameter of the OpenAI chat completions request.

The JSON schema is used to instruct the GPT-4o model to generate a strict output that adheres to the structure defined. The approach using Pydantic makes it easier for developers to manage the data structure in code, with helpful descriptions and examples that will be included in the final JSON schema.

Demonstrated below, you can see how the Invoice data model is understood by the OpenAI request:

In [24]:
# Highlight the schema sent to the OpenAI model
print(json.dumps(Invoice.model_json_schema(), indent=2))

{
  "$defs": {
    "InvoiceAddress": {
      "description": "A class representing an address in an invoice.\n\nAttributes:\n    street: Street address\n    city: City, e.g. New York\n    state: State, e.g. NY\n    postal_code: Postal code, e.g. 10001\n    country: Country, e.g. USA",
      "properties": {
        "street": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "Street address, e.g. 123 Main St.",
          "title": "Street"
        },
        "city": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "City, e.g. New York",
          "title": "City"
        },
        "state": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The Markdown representation of the document structure as determined by Azure AI Document Intelligence.
- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction by comparing against the Azure AI Document Intelligence analysis.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The side-by-side comparison of the expected output and the output generated by Azure OpenAI's GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [25]:
# Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(Markdown(markdown))

NEXGEN
Innovation drives progress


# INVOICE


<table>
<tr>
<td>Customer:</td>
<td colspan="2">White Triangle</td>
</tr>
<tr>
<td>Address:</td>
<td colspan="2">Unit 1, Fork Works, Rotherham, S1 1RT</td>
</tr>
<tr>
<td>Delivery Date:</td>
<td colspan="2">6/12/2024</td>
</tr>
<tr>
<td>Invoice #:</td>
<td>4315272</td>
<td>Purchase Order: 17153</td>
</tr>
</table>


PRODUCT ID

UNIT PRICE QUANTITY TOTAL PRICE


<table>
<tr>
<td>H509L</td>
<td>5.00</td>
<td>150</td>
<td>750.00</td>
</tr>
<tr>
<td>TOTAL</td>
<td></td>
<td>150</td>
<td>750.00</td>
</tr>
</table>


<table>
<tr>
<td>CUSTOMER SIGNATURE:</td>
<td>NAME:</td>
</tr>
<tr>
<td>DRIVER SIGNATURE:</td>
<td>NAME:</td>
</tr>
</table>


<!-- PageBreak -->

NEXGEN
Innovation drives progress

RETURNS


<table>
<tr>
<td>Customer:</td>
<td colspan="2">White Triangle</td>
</tr>
<tr>
<td>Address:</td>
<td colspan="2">Unit 1, Fork Works, Rotherham, S1 1RT</td>
</tr>
<tr>
<td>Delivery Date:</td>
<td colspan="2">6/12/2024</td>
</tr>
<tr>
<td>Invoice #:</td>
<td>4315272</td>
<td>Invoice #: 4315272</td>
</tr>
</table>


PRODUCT ID

QUANTITY REASON


<table>
<tr>
<td>H509M</td>
<td>150</td>
<td>Incorrect size. Too small.</td>
</tr>
<tr>
<td>TOTAL</td>
<td>150</td>
<td></td>
</tr>
</table>


<table>
<tr>
<td>CUSTOMER SIGNATURE:</td>
<td>NAME:</td>
</tr>
<tr>
<td>DRIVER SIGNATURE:</td>
<td>NAME:</td>
</tr>
</table>


In [26]:
# Gets the parsed Invoice object from the completion response.
invoice = completion.choices[0].message.parsed

# Determines the accuracy of the extracted data against the expected values.
accuracy = invoice_evaluator.evaluate(invoice)

# Determines the confidence of the extracted data against the expected values using the result of the Azure AI Document Intelligence pre-built layout analysis.
confidence = evaluate_confidence(invoice.to_dict(), result)

# Gets the total execution time of the data extraction process.
total_elapsed = di_stopwatch.elapsed + oai_stopwatch.elapsed

# Gets the prompt tokens and completion tokens from the completion response.
prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

# Save the output of the data extraction result.
extraction_result = DataExtractionResult(invoice.to_dict(), confidence, accuracy, prompt_tokens, completion_tokens, total_elapsed)

with open(f"{working_dir}/samples/extraction/text-based/document-intelligence-openai.{pdf_fname}.json", "w") as f:
    f.write(extraction_result.to_json(indent=4))
    
# Display the outputs of the data extraction process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Confidence": f"{confidence['_overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Document Intelligence Execution Time": f"{di_stopwatch.elapsed:.2f} seconds",
        "OpenAI Execution Time": f"{oai_stopwatch.elapsed:.2f} seconds",
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens
    }
])

display(Markdown(df.to_markdown(index=False, tablefmt='unsafehtml')))
display(Markdown(extraction_comparison(expected.to_dict(), invoice.to_dict(), confidence)))

<table>
<thead>
<tr><th>Accuracy  </th><th>Confidence  </th><th>Execution Time  </th><th>Document Intelligence Execution Time  </th><th>OpenAI Execution Time  </th><th style="text-align: right;">  Prompt Tokens</th><th style="text-align: right;">  Completion Tokens</th></tr>
</thead>
<tbody>
<tr><td>92.00%    </td><td>99.54%      </td><td>11.47 seconds   </td><td>5.16 seconds                          </td><td>6.31 seconds           </td><td style="text-align: right;">           2756</td><td style="text-align: right;">                293</td></tr>
</tbody>
</table>

<table>
<thead>
<tr><th>Field                               </th><th>Expected                  </th><th>Extracted                                                   </th><th style="text-align: right;">  Confidence</th></tr>
</thead>
<tbody>
<tr><td>customer_name                       </td><td>White Triangle            </td><td><span style='color: green'>White Triangle</span>            </td><td style="text-align: right;">       0.994</td></tr>
<tr><td>customer_address_street             </td><td>Unit 1, Fork Works        </td><td><span style='color: green'>Unit 1, Fork Works</span>        </td><td style="text-align: right;">       0.992</td></tr>
<tr><td>customer_address_city               </td><td>Rotherham                 </td><td><span style='color: green'>Rotherham</span>                 </td><td style="text-align: right;">       0.992</td></tr>
<tr><td>customer_address_state              </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>customer_address_postal_code        </td><td>S1 1RT                    </td><td><span style='color: green'>S1 1RT</span>                    </td><td style="text-align: right;">       0.992</td></tr>
<tr><td>customer_address_country            </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>customer_tax_id                     </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>shipping_address                    </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>purchase_order                      </td><td>17153                     </td><td><span style='color: green'>17153</span>                     </td><td style="text-align: right;">       0.997</td></tr>
<tr><td>invoice_id                          </td><td>4315272                   </td><td><span style='color: green'>4315272</span>                   </td><td style="text-align: right;">       0.997</td></tr>
<tr><td>invoice_date                        </td><td>2024-12-06                </td><td><span style='color: red'>None</span>                        </td><td style="text-align: right;">       0    </td></tr>
<tr><td>payable_by                          </td><td>                          </td><td><span style='color: red'>2024-12-06</span>                  </td><td style="text-align: right;">       0    </td></tr>
<tr><td>vendor_name                         </td><td>NEXGEN                    </td><td><span style='color: green'>NEXGEN</span>                    </td><td style="text-align: right;">       0.996</td></tr>
<tr><td>vendor_address                      </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>vendor_tax_id                       </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>remittance_address                  </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>subtotal                            </td><td>750.0                     </td><td><span style='color: green'>750.0</span>                     </td><td style="text-align: right;">       0.998</td></tr>
<tr><td>total_discount                      </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>total_tax                           </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>invoice_total                       </td><td>750.0                     </td><td><span style='color: green'>750.0</span>                     </td><td style="text-align: right;">       0.998</td></tr>
<tr><td>payment_terms                       </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>products_0_product_code             </td><td>H509L                     </td><td><span style='color: green'>H509L</span>                     </td><td style="text-align: right;">       0.994</td></tr>
<tr><td>products_0_description              </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>products_0_quantity                 </td><td>150                       </td><td><span style='color: green'>150</span>                       </td><td style="text-align: right;">       0.999</td></tr>
<tr><td>products_0_tax                      </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>products_0_tax_rate                 </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>products_0_unit_price               </td><td>5.0                       </td><td><span style='color: green'>5.0</span>                       </td><td style="text-align: right;">       0.992</td></tr>
<tr><td>products_0_total                    </td><td>750.0                     </td><td><span style='color: green'>750.0</span>                     </td><td style="text-align: right;">       0.998</td></tr>
<tr><td>products_0_reason                   </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>total_item_quantity                 </td><td>150.0                     </td><td><span style='color: green'>150.0</span>                     </td><td style="text-align: right;">       0    </td></tr>
<tr><td>items_customer_signature_signatory  </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>items_customer_signature_is_signed  </td><td>False                     </td><td><span style='color: green'>False</span>                     </td><td style="text-align: right;">       0    </td></tr>
<tr><td>items_vendor_signature_signatory    </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>items_vendor_signature_is_signed    </td><td>False                     </td><td><span style='color: green'>False</span>                     </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_0_product_code              </td><td>H509M                     </td><td><span style='color: green'>H509M</span>                     </td><td style="text-align: right;">       0.995</td></tr>
<tr><td>returns_0_description               </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_0_quantity                  </td><td>150                       </td><td><span style='color: green'>150</span>                       </td><td style="text-align: right;">       0.999</td></tr>
<tr><td>returns_0_tax                       </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_0_tax_rate                  </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_0_unit_price                </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_0_total                     </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_0_reason                    </td><td>Incorrect size. Too small.</td><td><span style='color: green'>Incorrect size. Too small.</span></td><td style="text-align: right;">       0.993</td></tr>
<tr><td>total_return_quantity               </td><td>150.0                     </td><td><span style='color: green'>150.0</span>                     </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_customer_signature_signatory</td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_customer_signature_is_signed</td><td>False                     </td><td><span style='color: green'>False</span>                     </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_vendor_signature_signatory  </td><td>                          </td><td><span style='color: green'>None</span>                      </td><td style="text-align: right;">       0    </td></tr>
<tr><td>returns_vendor_signature_is_signed  </td><td>False                     </td><td><span style='color: green'>False</span>                     </td><td style="text-align: right;">       0    </td></tr>
</tbody>
</table>